In [40]:
import osmnx as ox
import pandas as pd
import geopandas as gpd
import networkx as nx
from shapely.geometry import Point, LineString
import matplotlib.pyplot as plt
import datetime
from shapely.geometry import Polygon, MultiPolygon
%matplotlib inline

In [41]:
ox.config(data_folder='Data', logs_folder='logs', 
          imgs_folder='imgs', cache_folder='cache',
          use_cache=True, log_console=True,log_name='osmnx', 
          log_file=True, log_filename='osmnx')

crs_osm = {'init':'epsg:4326'}           #crs that osm uses

In [42]:
def load_data(name):
    area = gpd.read_file('Data/{}/{}_shape/'.format(name, name))
    area = ox.project_gdf(area, to_crs=crs_osm)
    geometry = area.unary_union
    pois = ox.pois_from_polygon(geometry, amenities=['bar','pub','nightclub'])
    pois = ox.project_gdf(pois, to_crs=crs_osm)

    for item in ['way', 'relation']:
        pois.loc[pois.element_type==item, 'geometry'] = \
            pois[pois.element_type==item]['geometry'].map(lambda x: x.centroid)

    pois['geometry_type'] = pois.geometry.map(type)

    s=pois['geometry']
    pois['x'] = s.apply(lambda p: p.x)
    pois['y'] = s.apply(lambda p: p.y)

    dat_fin = gpd.sjoin(pois, area, op = 'within')
    pois = dat_fin

    G = ox.load_graphml('{}/{}_walk.graphml'.format(name,name))
    G = ox.project_graph(G, to_crs=crs_osm)
    return G, pois

In [48]:
def make_plot(G, pois):
    fig, ax = ox.plot_graph(G, node_color='#aaaaaa', node_size=0, show=False, close=True, fig_height=20, edge_linewidth=0.1, bgcolor='black',edge_color='#969494')
    ax.scatter(pois.loc[pois['amenity'] == 'pub']['x'],pois.loc[pois['amenity'] == 'pub']['y'], zorder=3, c='magenta', alpha=0.5, marker='o',s=30)
    ax.scatter(pois.loc[pois['amenity'] == 'bar']['x'],pois.loc[pois['amenity'] == 'bar']['y'], zorder=3, c='yellow', alpha=0.5, marker='o',s=30)
    ax.scatter(pois.loc[pois['amenity'] == 'nightclub']['x'],pois.loc[pois['amenity'] == 'nightclub']['y'], zorder=3, c='cyan', alpha=0.5, marker='o',s=30)
    suptitle_font = {'family':'DejaVu Sans', 'fontsize':45, 'fontweight':'normal', 'y':0.15, 'color':'white'}
    fig.canvas.draw()
    fig.savefig('imgs/{}_nightlife.png'.format(name),transparent=True, dpi=300)
    fig

In [46]:
name = 'Manhattan'
G, pois = load_data(name)
make_plot(G, pois)

/anaconda3/lib/python3.6/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
